In [ ]:
from webscraper.page_scraper import scrap_singer_page, scrap_song_page, scrap_album_page, initialize_driver
from webscraper.ten_years_chart_scraper import ten_year
from webscraper.year_chart_scraper import yearchart
import pymysql

In [ ]:
"""
본인 세팅 맞추기
"""
USER = 'dbuser'
PWD = 'dbuserdbuser'
HOST = 'localhost'
DB = 'MELON'
CHARSET='utf8'


"""
연대차트 2000, 2010년대 스크랩
"""

db = pymysql.connect(host=HOST, port=3306, user=USER, passwd=PWD, db=DB, charset=CHARSET)
cursor = db.cursor()
driver = initialize_driver()

for decade in [2000, 2010]:
    result = ten_year(decade, driver='./webscraper/chromedriver') ## WINDOWS driver 없애기
    rank_song_id_album_id_list = result[0]
    
    for rank, song_id, album_id in rank_song_id_album_id_list:
        
        s_id = int(song_id.split("'")[1])
        
        # scrap_song_page불러서 얻은걸로 song 집어넣고
        song_result = scrap_song_page(s_id, driver)
        
        sql = """
        INSERT INTO SONG VALUES(
            {}, "{}", "{}", {}, {}
        ) ON DUPLICATE KEY UPDATE ID = ID;
        """.format(s_id, song_result['name'], song_result['genre'], song_result['num_like'], song_result['num_comment'])
        cursor.execute(sql)
        db.commit()
        
        # song-singer 에 넣으려면 singer도 받아서 singer테이블에 넣기
        singer_list = song_result['singers']
        for singer_id in singer_list:
            singer_result = scrap_singer_page(singer_id, driver)
            
            if singer_result['debut_date'] is not None:
                debut_date = singer_result['debut_date'].replace('.', '-')
                if len(debut_date) == 4:
                    debut_date = debut_date + '.01.01'
                elif len(debut_date) == 7:
                    debut_date = debut_date + '.01'
            else:
                debut_date = '-'
            
            # scrap_singer_page: id, name, artist_type, company, debut_date, num_fan
            sql = """
            INSERT INTO SINGER VALUES(
                {0},"{1}", IF("{2}" = "-", NULL, "{2}"), "{3}", {4}
            ) ON DUPLICATE KEY UPDATE ID = ID;
            """.format(singer_result['id'], singer_result['name'], debut_date, singer_result['company'], singer_result['num_fan'])
            cursor.execute(sql)
            db.commit()
            
            # song-singer table
            sql = """
            INSERT INTO SONG_SINGER VALUES (
                {},{}
            ) ON DUPLICATE KEY UPDATE SONG_ID = SONG_ID;
            """.format(s_id, singer_id)
            cursor.execute(sql)
            db.commit()
    
        # ten_year_chart
        sql = """
        INSERT INTO TEN_YEARS_CHART VALUES(
            MAKEDATE({}, 1),{},{}
        ) ON DUPLICATE KEY UPDATE START_YEAR=START_YEAR;
        """.format(decade, rank, s_id)
        cursor.execute(sql)
        db.commit()
        
        # scrap_album_page: 
        #{id, name,release_date, num_like, rating, num_rating, num_comment}
        #artist_id_list, song_id_list
        a_id = int(album_id.split("'")[1])
        album_result = scrap_album_page(a_id, driver)
        album_info = album_result[0]
        
        release_date = album_info['release_date'].replace('.', '-')
        if len(release_date) == 4:
            release_date = release_date + '-01-01'
        elif len(release_date) == 7:
            release_date = release_date + '-01'
        
        sql = """
        INSERT INTO ALBUM VALUES (
            {}, "{}", "{}", {}, {}, {}, {}
        ) ON DUPLICATE KEY UPDATE ID=ID
        """.format(a_id, album_info['name'], release_date, album_info['num_like'], album_info['rating'], album_info['num_rating'], album_info['num_comment'])

        cursor.execute(sql)
        db.commit()
        
        # album_song
        sql = """
        INSERT INTO ALBUM_SONG VALUES(
            {},{}
        ) ON DUPLICATE KEY UPDATE ALBUM_ID = ALBUM_ID
        """.format(a_id, s_id)
        cursor.execute(sql)
        db.commit()
        
        for i in album_result[1]:
            singer_result = scrap_singer_page(i, driver)
            
            if singer_result['debut_date'] is None:
                debut_date = '-'
            else:
                debut_date = singer_result['debut_date'].replace('.', '-')
                if len(debut_date) == 4:
                    debut_date = debut_date + '-01-01'
                elif len(debut_date) == 7:
                    debut_date = debut_date + '-01'
            
            # scrap_singer_page: id, name, artist_type, company, debut_date, num_fan
            sql = """
            INSERT INTO SINGER VALUES(
                {0},"{1}", IF("{2}" = "-", NULL, "{2}"), "{3}", {4}
            ) ON DUPLICATE KEY UPDATE ID = ID;
            """.format(singer_result['id'], singer_result['name'], debut_date, singer_result['company'], singer_result['num_fan'])
            cursor.execute(sql)
            db.commit()
            
            # album_singer
            sql = """
            INSERT INTO ALBUM_SINGER VALUES(
                {}, {}
            ) ON DUPLICATE KEY UPDATE ALBUM_ID=ALBUM_ID
            """.format(a_id, i)
            cursor.execute(sql)
            db.commit()
cursor.close()
db.close()
driver.quit()
                

In [ ]:
db = pymysql.connect(host=HOST, port=3306, user=USER, passwd=PWD, db=DB, charset=CHARSET)
cursor = db.cursor()
driver = initialize_driver()

# [(순위, 곡아이디, 좋아요 수)], [가수아이디], [앨범아이디]
for year in range(2000, 2020):
    result = yearchart(year, './webscraper/chromedriver') # WINDOWS driver 없애기
    rank_song_like = result[0]
    albums = result[2]
    ranks = [r[0] for r in rank_song_like]
    songs = [r[1] for r in rank_song_like]
    
    for rank, song_id, album_id in zip(ranks, songs, albums):
        s_id = int(song_id.split("'")[1])
        
        # scrap_song_page불러서 얻은걸로 song 집어넣고
        song_result = scrap_song_page(s_id, driver)
        
        sql = """
        INSERT INTO SONG VALUES(
            {}, "{}", "{}", {}, {}
        ) ON DUPLICATE KEY UPDATE ID = ID;
        """.format(s_id, song_result['name'], song_result['genre'], song_result['num_like'], song_result['num_comment'])
        cursor.execute(sql)
        db.commit()
        
        # song-singer 에 넣으려면 singer도 받아서 singer테이블에 넣기
        singer_list = song_result['singers']
        for singer_id in singer_list:
            singer_result = scrap_singer_page(singer_id, driver)
            
            if singer_result['debut_date'] is None:
                debut_date = '-'
            else:
                debut_date = singer_result['debut_date'].replace('.', '-')
                if len(debut_date) == 4:
                    debut_date = debut_date + '-01-01'
                elif len(debut_date) == 7:
                    debut_date = debut_date + '-01'
            
            # scrap_singer_page: id, name, artist_type, company, debut_date, num_fan
            sql = """
            INSERT INTO SINGER VALUES(
                {0},"{1}", IF("{2}" = "-", NULL, "{2}"), "{3}", {4}
            ) ON DUPLICATE KEY UPDATE ID = ID;
            """.format(singer_result['id'], singer_result['name'], debut_date, singer_result['company'], singer_result['num_fan'])
            cursor.execute(sql)
            db.commit()
            
            # song-singer table
            sql = """
            INSERT INTO SONG_SINGER VALUES (
                {},{}
            ) ON DUPLICATE KEY UPDATE SONG_ID = SONG_ID;
            """.format(s_id, singer_id)
            cursor.execute(sql)
            db.commit()
            
        # year_chart insert
        sql = """
        INSERT INTO YEAR_CHART VALUES (
            MAKEDATE({}, 1), {}, {}
        ) ON DUPLICATE KEY UPDATE SONG_ID = SONG_ID
        """.format(year, rank, s_id)
        
        cursor.execute(sql)
        db.commit()
        
        # 앨범 리스트

        a_id = int(album_id.split("'")[1])
        album_result = scrap_album_page(a_id, driver)
        album_info = album_result[0]
        
        release_date = album_info['release_date'].replace('.', '-')
        if len(release_date) == 4:
            release_date = release_date + '-01-01'
        elif len(release_date) == 7:
            release_date = release_date + '-01'
        
        sql = """
        INSERT INTO ALBUM VALUES (
            {}, "{}", "{}", {}, {}, {}, {}
        ) ON DUPLICATE KEY UPDATE ID = ID
        """.format(a_id, album_info['name'], release_date, album_info['num_like'], album_info['rating'], album_info['num_rating'], album_info['num_comment'])
        cursor.execute(sql)
        db.commit()
        
        # album_song
        sql = """
        INSERT INTO ALBUM_SONG VALUES(
             {},{}
        ) ON DUPLICATE KEY UPDATE ALBUM_ID = ALBUM_ID
        """.format(a_id, s_id)
        cursor.execute(sql)
        db.commit()
        
        
        
        for i in album_result[1]:
            singer_result = scrap_singer_page(i, driver)
            if singer_result['debut_date'] is None:
                debut_date = '-'
            else:
                debut_date = singer_result['debut_date'].replace('.', '-')
                if len(debut_date) == 4:
                    debut_date = debut_date + '-01-01'
                elif len(debut_date) == 7:
                    debut_date = debut_date + '-01'
            
            # scrap_singer_page: id, name, artist_type, company, debut_date, num_fan
            sql = """
            INSERT INTO SINGER VALUES(
                {0},"{1}", IF("{2}" = "-", NULL, "{2}"), "{3}", {4}
            ) ON DUPLICATE KEY UPDATE ID = ID;
            """.format(singer_result['id'], singer_result['name'], debut_date, singer_result['company'], singer_result['num_fan'])
            cursor.execute(sql)
            db.commit()
            
            # album_singer
            sql = """
            INSERT INTO ALBUM_SINGER VALUES(
                {}, {}
            ) ON DUPLICATE KEY UPDATE ALBUM_ID = ALBUM_ID
            """.format(a_id, i)
            cursor.execute(sql)
            db.commit()        

cursor.close()
db.close()
driver.quit()